In [1]:
#!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
#!tar -xf aclImdb_v1.tar.gz
#!rm -r aclImdb/train/unsup
!cat aclImdb/train/pos/4077_10.txt

I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drink and food in reach so you don't have to stop the film.<br /><br />Enjoy

In [2]:
import os, pathlib, shutil, random

base_dir = pathlib.Path('aclImdb')
val_dir = base_dir / 'val'
train_dir = base_dir / 'train'
for category in ('neg', 'pos'):
    os.makedirs(val_dir / category)
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files)
    n_val_samples = int(0.2 * len(files))
    val_files = files[-n_val_samples:]
    for filename in val_files:
        shutil.move(train_dir / category / filename, val_dir / category / filename)

FileExistsError: [Errno 17] File exists: 'aclImdb/val/neg'

In [3]:
from tensorflow import keras
batch_size = 32

train_ds = keras.utils.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size
)

val_ds = keras.utils.text_dataset_from_directory(
    'aclImdb/val', batch_size=batch_size
)

test_ds = keras.utils.text_dataset_from_directory(
    'aclImdb/test', batch_size=batch_size
)

2023-09-20 16:51:55.822951: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-20 16:51:55.844916: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-20 16:51:55.845362: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-20 16:51:56.325097: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


2023-09-20 16:51:57.074229: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-20 16:51:57.074480: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Found 25000 files belonging to 2 classes.


In [4]:
for inputs, targets in train_ds:
    print(inputs.shape, inputs.dtype, inputs[0])
    print(targets.shape, targets.dtype, targets[0])
    break

(32,) <dtype: 'string'> tf.Tensor(b'I first saw this movie when I was a little kid and fell in love with it at once. The sets are breath taking and some of the script is damn right hilarious: "You sons of a thousand fleas".<br /><br />It is always shown on TV late at night or really early in the morning i woke up at about 3:00 am once and it had just started. TV companys need to show a little more respect and put it on prime time Sunday so everyone can get a chance to view this fine work.<br /><br />10/10', shape=(), dtype=string)
(32,) <dtype: 'int32'> tf.Tensor(1, shape=(), dtype=int32)


In [5]:
from keras import layers

text_vectorization = layers.TextVectorization(max_tokens=20000, output_mode='multi_hot')
text_only_train_ds = train_ds.map(lambda x, y: x)
text_vectorization.adapt(text_only_train_ds)

binary_1gram_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=16)
binary_1gram_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=16)
binary_1gram_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=16)

In [6]:
for inputs, targets in binary_1gram_train_ds:
    print(inputs.shape, inputs.dtype, inputs[0])
    print(targets.shape, targets.dtype, targets[0])
    break

(32, 20000) <dtype: 'float32'> tf.Tensor([1. 1. 1. ... 0. 0. 0.], shape=(20000,), dtype=float32)
(32,) <dtype: 'int32'> tf.Tensor(1, shape=(), dtype=int32)


In [7]:
import keras
from keras import layers, activations, optimizers, losses, metrics, callbacks

def get_model(max_tokens=20000, hidden_dim=16):
    inputs = layers.Input(shape=(max_tokens,))
    x = layers.Dense(hidden_dim, activation=activations.relu)(inputs)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation=activations.sigmoid)(x)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer=optimizers.RMSprop(), loss=losses.BinaryCrossentropy(), metrics=[metrics.BinaryAccuracy()])
    return model

In [9]:
model = get_model()
model.summary()
callback_list = [
    callbacks.ModelCheckpoint('binary_1gram', save_best_only=True)
]

model.fit(binary_1gram_train_ds, epochs=10, validation_data=binary_1gram_val_ds, callbacks=callback_list)

model = keras.models.load_model('binary_1gram')
print(model.evaluate(binary_1gram_test_ds))

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense_2 (Dense)             (None, 16)                320016    
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320033 (1.22 MB)
Trainable params: 320033 (1.22 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Epoch 1/10
600/625 [===========================>..] - ETA: 0s - loss: 0.4088 - binary_accuracy: 0.8265INFO:tensorflow:Assets written to: binary_1gram/assets


INFO:tensorflow:Assets written to: binary_1gram/assets


625/625 [==============================] - 1s 2ms/step - loss: 0.4073 - binary_accuracy: 0.8274 - val_loss: 0.2833 - val_binary_accuracy: 0.8918
Epoch 2/10
603/625 [===========================>..] - ETA: 0s - loss: 0.2706 - binary_accuracy: 0.8981INFO:tensorflow:Assets written to: binary_1gram/assets


INFO:tensorflow:Assets written to: binary_1gram/assets


625/625 [==============================] - 1s 2ms/step - loss: 0.2710 - binary_accuracy: 0.8979 - val_loss: 0.2769 - val_binary_accuracy: 0.8908
Epoch 3/10
625/625 [==============================] - 1s 1ms/step - loss: 0.2427 - binary_accuracy: 0.9153 - val_loss: 0.2853 - val_binary_accuracy: 0.8954
Epoch 4/10
625/625 [==============================] - 1s 1ms/step - loss: 0.2295 - binary_accuracy: 0.9216 - val_loss: 0.3016 - val_binary_accuracy: 0.8928
Epoch 5/10
625/625 [==============================] - 1s 2ms/step - loss: 0.2167 - binary_accuracy: 0.9275 - val_loss: 0.3172 - val_binary_accuracy: 0.8926
Epoch 6/10
625/625 [==============================] - 1s 2ms/step - loss: 0.2096 - binary_accuracy: 0.9318 - val_loss: 0.3268 - val_binary_accuracy: 0.8910
Epoch 7/10
625/625 [==============================] - 1s 1ms/step - loss: 0.2018 - binary_accuracy: 0.9351 - val_loss: 0.3394 - val_binary_accuracy: 0.8904
Epoch 8/10
625/625 [==============================] - 1s 1ms/step - loss: 0

In [13]:
text_vectorization = layers.TextVectorization(ngrams=2, max_tokens=20000, output_mode='multi_hot')
text_vectorization.adapt(text_only_train_ds)

In [16]:
binary_2gram_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=16)
binary_2gram_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=16)
binary_2gram_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=16)

model = get_model()
callback_list = [
    callbacks.ModelCheckpoint('binary_2gram', save_best_only=True)
]

model.fit(binary_2gram_train_ds, validation_data=binary_2gram_val_ds, epochs=10, callbacks=callback_list)

model = keras.models.load_model('binary_2gram')
print(model.evaluate(binary_2gram_test_ds))

Epoch 1/10
599/625 [===========================>..] - ETA: 0s - loss: 0.3719 - binary_accuracy: 0.8458INFO:tensorflow:Assets written to: binary_2gram/assets


INFO:tensorflow:Assets written to: binary_2gram/assets


625/625 [==============================] - 1s 2ms/step - loss: 0.3708 - binary_accuracy: 0.8467 - val_loss: 0.2743 - val_binary_accuracy: 0.8948
Epoch 2/10
602/625 [===========================>..] - ETA: 0s - loss: 0.2452 - binary_accuracy: 0.9112INFO:tensorflow:Assets written to: binary_2gram/assets


INFO:tensorflow:Assets written to: binary_2gram/assets


625/625 [==============================] - 1s 2ms/step - loss: 0.2451 - binary_accuracy: 0.9111 - val_loss: 0.2733 - val_binary_accuracy: 0.9002
Epoch 3/10
625/625 [==============================] - 1s 1ms/step - loss: 0.2017 - binary_accuracy: 0.9330 - val_loss: 0.2878 - val_binary_accuracy: 0.9010
Epoch 4/10
625/625 [==============================] - 1s 1ms/step - loss: 0.1856 - binary_accuracy: 0.9390 - val_loss: 0.3101 - val_binary_accuracy: 0.8984
Epoch 5/10
625/625 [==============================] - 1s 1ms/step - loss: 0.1793 - binary_accuracy: 0.9463 - val_loss: 0.3292 - val_binary_accuracy: 0.8962
Epoch 6/10
625/625 [==============================] - 1s 1ms/step - loss: 0.1746 - binary_accuracy: 0.9489 - val_loss: 0.3388 - val_binary_accuracy: 0.8976
Epoch 7/10
625/625 [==============================] - 1s 1ms/step - loss: 0.1667 - binary_accuracy: 0.9526 - val_loss: 0.3618 - val_binary_accuracy: 0.8938
Epoch 8/10
625/625 [==============================] - 1s 2ms/step - loss: 0

In [17]:
text_vectorization = layers.TextVectorization(ngrams=2, max_tokens=20000, output_mode='tf_idf')
text_vectorization.adapt(text_only_train_ds)

In [19]:
tfidf_2gram_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=16)
tfidf_2gram_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=16)
tfidf_2gram_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=16)

callback_list = [
    callbacks.ModelCheckpoint('tfidf_2gram', save_best_only=True)
]

model = get_model()
model.fit(tfidf_2gram_train_ds.cache(), validation_data=tfidf_2gram_val_ds, epochs=10, callbacks=callback_list)

model = keras.models.load_model('tfidf_2gram')
print(model.evaluate(tfidf_2gram_test_ds))

Epoch 1/10
615/625 [============================>.] - ETA: 0s - loss: 0.4980 - binary_accuracy: 0.7779INFO:tensorflow:Assets written to: tfidf_2gram/assets


INFO:tensorflow:Assets written to: tfidf_2gram/assets


625/625 [==============================] - 1s 2ms/step - loss: 0.4956 - binary_accuracy: 0.7785 - val_loss: 0.2976 - val_binary_accuracy: 0.8944
Epoch 2/10
625/625 [==============================] - 1s 1ms/step - loss: 0.3299 - binary_accuracy: 0.8676 - val_loss: 0.3101 - val_binary_accuracy: 0.8844
Epoch 3/10
625/625 [==============================] - 1s 1ms/step - loss: 0.2841 - binary_accuracy: 0.8899 - val_loss: 0.3039 - val_binary_accuracy: 0.8970
Epoch 4/10
625/625 [==============================] - 1s 1ms/step - loss: 0.2622 - binary_accuracy: 0.8992 - val_loss: 0.3261 - val_binary_accuracy: 0.8810
Epoch 5/10
625/625 [==============================] - 1s 1ms/step - loss: 0.2476 - binary_accuracy: 0.9043 - val_loss: 0.3430 - val_binary_accuracy: 0.8780
Epoch 6/10
625/625 [==============================] - 1s 1ms/step - loss: 0.2363 - binary_accuracy: 0.9053 - val_loss: 0.4036 - val_binary_accuracy: 0.8610
Epoch 7/10
625/625 [==============================] - 1s 1ms/step - loss: 0